# LangChain Expression Language

In [ ]:
// import { ChatOpenAI } from "@langchain/openai";
import { ChatOllama } from "@langchain/community/chat_models/ollama";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";

const prompt = ChatPromptTemplate.fromMessages([
  ["human", "Tell me a short joke about {topic}"],
]);
// const model = new ChatOpenAI({});
const model = new ChatOllama({
  baseUrl: "http://localhost:11434", // Default value
  model: "qwen", // Default value
  // model: "llama3", // Default value
  // model: "datouxia/llama3-8b-chinese-chat-q8-v2", // Default value
});
const outputParser = new StringOutputParser();

const chain = prompt.pipe(model).pipe(outputParser);

const response = await chain.invoke({
  topic: "ice cream",
});
console.log(response);
/**
Why did the ice cream go to the gym?
Because it wanted to get a little "cone"ditioning!
 */

## Prompt
lc_serializable: 这是一个标记字段,表示该对象可以被序列化。

lc_kwargs: 这是一个字典,包含了用于实例化该对象的关键字参数。在这个例子中,它包含了一个messages字段,这是一个消息列表,代表对话历史。

lc_namespace: 这是一个列表,表示该对象所属的命名空间。在这个例子中,它来自langchain_core.prompt_values模块。

In [ ]:
const prompt = ChatPromptTemplate.fromMessages([
  ["human", "Tell me a short joke about {topic}"],
]);
const promptValue = await prompt.invoke({ topic: "ice cream" });
console.log(promptValue);
/**
ChatPromptValue {
  messages: [
    HumanMessage {
      content: 'Tell me a short joke about ice cream',
      name: undefined,
      additional_kwargs: {}
    }
  ]
}
 */
const promptAsMessages = promptValue.toChatMessages();
console.log(promptAsMessages);
/**
[
  HumanMessage {
    content: 'Tell me a short joke about ice cream',
    name: undefined,
    additional_kwargs: {}
  }
]
 */
const promptAsString = promptValue.toString();
console.log(promptAsString);
/**
Human: Tell me a short joke about ice cream
 */

## Model

In [ ]:
import { ChatOllama } from "@langchain/community/chat_models/ollama";
const model = new ChatOllama({
  baseUrl: "http://localhost:11434", // Default value
  model: "qwen", // Default value
});
const promptAsString = "Human: Tell me a short joke about ice cream";

const response = await model.invoke(promptAsString);
console.log(response);
/**
AIMessage {
  content: 'Sure, here you go: Why did the ice cream go to school? Because it wanted to get a little "sundae" education!',
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}
 */

### If our model was an LLM, it would output a string.

In [ ]:
import { Ollama } from "@langchain/community/llms/ollama";

const model = new Ollama({
  baseUrl: "http://localhost:11434", // Default value
  model: "qwen", // Default value
});
const promptAsString = "Human: Tell me a short joke about ice cream";

const response = await model.invoke(promptAsString);
console.log(response);
/**
Why did the ice cream go to therapy?

Because it was feeling a little rocky road.
 */

### ChatOllama 和 Ollama 区别
ChatOllama 和 Ollama 是两个不同的概念,它们在 LangChain 中具有不同的用途和含义。

Ollama

Ollama 是一个 LangChain 中的基础模型类,它继承自 BaseModel 类,旨在为语言模型提供统一的接口。Ollama 类本身并不包含任何模型实现,而是作为子类化的基类,被其他具体的语言模型类继承和扩展。
Ollama 类定义了一些通用的方法,如 generate 用于生成文本,query 用于查询模型,以及一些辅助方法。通过继承 Ollama 类,具体的语言模型实现可以复用这些通用方法,并针对自身的特性进行扩展和定制。

ChatOllama

ChatOllama 是 LangChain 中一个专门用于对话模型的类,它继承自 Ollama 类。ChatOllama 类针对对话场景进行了优化和扩展,提供了更加适合对话的接口和功能。
ChatOllama 类的主要特点包括:

支持维护对话历史记录,以便模型能够根据上下文生成响应。
提供了 predict 方法,用于生成对话响应。
支持设置不同的对话策略,如提示词、开头和结尾等。
可以与不同的对话格式和数据结构集成,如 ChatPromptValue、ChatResult 等。

总的来说,Ollama 是一个通用的基础模型类,而 ChatOllama 则是针对对话场景的特殊化模型类,两者的关系是继承关系。在构建对话应用程序时,通常会使用 ChatOllama 或其他继承自 ChatOllama 的具体对话模型实现。

## RAG Search Example

In [13]:
// import { ChatOpenAI, OpenAIEmbeddings } from "@langchain/openai";
import { OllamaEmbeddings } from "@langchain/community/embeddings/ollama";
import { ChatOllama } from "@langchain/community/chat_models/ollama";
// import { HNSWLib } from "@langchain/community/vectorstores/hnswlib";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { Document } from "@langchain/core/documents";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import {
  RunnableLambda,
  RunnableMap,
  RunnablePassthrough,
} from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const vectorStore = await MemoryVectorStore.fromDocuments(
  [
    new Document({ pageContent: "Harrison worked at Kensho" }),
    new Document({ pageContent: "Bears like to eat honey." }),
  ],
  new OllamaEmbeddings({
    model: "nomic-embed-text", // 成功
    // model: "mxbai-embed-large", // 成功
    // model: "snowflake-arctic-embed", // 失败
    baseUrl: "http://localhost:11434", // default value
  })
);
const retriever = vectorStore.asRetriever(1);

const prompt = ChatPromptTemplate.fromMessages([
  [
    "ai",
    `Answer the question based on only the following context:
  
{context}`,
  ],
  ["human", "{question}"],
]);
// const model = new ChatOpenAI({});
const model = new ChatOllama({
  baseUrl: "http://localhost:11434", // Default value
  model: "qwen", // Default value
  verbose: true,
});
const outputParser = new StringOutputParser();

const setupAndRetrieval = RunnableMap.from({
  context: new RunnableLambda({
    func: (input: string) => {
        console.log(input)
      return retriever.invoke(input).then((response) => {
          console.log(response)
          return response[0].pageContent
      }) 
    }
  }).withConfig({ runName: "contextRetriever" }),
  question: new RunnablePassthrough(),
});
const chain = setupAndRetrieval.pipe(prompt).pipe(model).pipe(outputParser);

const response = await chain.invoke("Where did Harrison work?");
console.log(response);
/**
Harrison worked at Kensho.
 */

Where did Harrison work?
[ Document { pageContent: "Harrison worked at Kensho", metadata: {} } ]
[llm/start] [1:llm:ChatOllama] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "AIMessage"
        ],
        "kwargs": {
          "content": "Answer the question based on only the following context:\n  \nHarrison worked at Kensho",
          "tool_calls": [],
          "invalid_tool_calls": [],
          "additional_kwargs": {},
          "response_metadata": {}
        }
      },
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "Where did Harrison work?",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:llm:ChatOllama] [4.30s] Exiting LLM run w